In [2]:
from domino_data.vectordb import DominoPineconeConfiguration
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Pinecone
from langchain import PromptTemplate
from langchain_community.chat_models import ChatMlflow

import csv
import os
import pinecone
import sys

from mlflow.deployments import get_deploy_client
import os

client = get_deploy_client(os.environ['DOMINO_MLFLOW_DEPLOYMENTS'])

In [3]:
# Load the embedding model
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding_model_name = "BAAI/bge-small-en"
os.environ['SENTENCE_TRANSFORMERS_HOME'] = './model_cache/'
embeddings = HuggingFaceBgeEmbeddings(model_name=embedding_model_name,
                                      model_kwargs=model_kwargs,
                                      encode_kwargs=encode_kwargs
                                     )

In [4]:
PINECONE_ENV="gcp-starter"

In [5]:
# Domino Pinecone Data Source name 
datasource_name = "mrag-fin-docs-ja"

# Load Domino Pinecone Data Source Configuration 
conf = DominoPineconeConfiguration(datasource=datasource_name)

# api_key variable is a mandatory non-empty field used for 
# the native pinecone python client initialization 
# using Pinecone Data Source name 
api_key = os.environ.get("DOMINO_VECTOR_DB_METADATA", datasource_name)

# initialize pinecone
pinecone.init(
    api_key=api_key, 
    environment=PINECONE_ENV,
    openapi_config=conf # Domino Pinecone Data Source Configuration 
)

# Load Pinecone Index
index_name = "mrag-fin-docs"
index = pinecone.Index(index_name)
text_field = "text"

# switch back to normal index for langchain
vectorstore = Pinecone(
    index, embeddings.embed_query, text_field # Using embedded data from Domino AI Gateway Endpoint
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.vectorstores.pinecone.Pinecone` was deprecated in langchain-community 0.0.18 and will be removed in 0.2.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:68: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [6]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.00419,
 'namespaces': {'': {'vector_count': 419}},
 'total_vector_count': 419}

In [31]:

prompt_template = """You are an AI assistant with expertise in financial analysis, your task is to dissect corporate filings (e.g., 10-K, 10-Q, 8-K reports) of publicly traded companies and provide detailed, accurate responses to the following user question about the company's financial health, market position, and future prospects. 
When interacting, adhere to the following guidelines:
You are given the following extracted parts of a long document and a question. 
If you don't know the answer, just say "Hmm, I'm not sure." Don't try to make up an answer.
If the question is not related to corporate filings analysis, politely inform them that you are tuned to only answer questions pertaining to corporate filings analysis.
Your primary objective is to deliver insightful, accurate, and helpful answers that enable users to make informed decisions based on corporate filings analysis. Each response should be tailored to the user's question, enhancing understanding of the company's financial status and strategic direction. pertaining to policy coverage.
Question: {question}
=========
{context}
=========
Answer in Markdown:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["question", "context"])
#
chain_type_kwargs = {"prompt": PROMPT}

In [33]:
rag_llm = ChatMlflow(
        target_uri=os.environ["DOMINO_MLFLOW_DEPLOYMENTS"],
        endpoint="chat-gpt35turbo-ja", 
        temperature=0.0,
    )

In [34]:

qa_chain = RetrievalQA.from_chain_type(llm=rag_llm, # AI Gateway Endpoint
                                       chain_type="stuff",
                                       chain_type_kwargs={"prompt": PROMPT},
                                       retriever=vectorstore.as_retriever(search_kwargs={"k": 5}), # Domino Pinecone Data Source
                                       return_source_documents=False
                                      )

In [36]:
user_question = input("Please ask your financial question:")
result = qa_chain(user_question)

Please ask your financial question: How much revenue did Apple generate in FY23?


In [37]:
print(result)

{'query': 'How much revenue did Apple generate in FY23?', 'result': 'To determine the revenue generated by Apple in FY23, we can look at the "CONSOLIDATED STATEMENTS OF OPERATIONS" section of the 2023 Form 10-K report. In FY23, Apple reported total net sales of $383,285 million, which is the sum of product sales ($298,085 million) and service sales ($85,200 million). Therefore, Apple generated $383,285 million in revenue in FY23.'}
